In [359]:
import pandas as pd
import plotnine as pn
from plotnine import aes, ggplot
import pathlib
from datetime import datetime
from prophet import Prophet

# Import Datasets
The following datasets are used:
- cpi file as supplied by zindi
- currency as supplied by zindi
- jse data as supplied by zindi
-  Monthly credit detail excel released by the reserve bank (https://www.resbank.co.za/en/home/what-we-do/statistics/releases/selected-statistics)
- fuel prices pulled from open price engin

In [2]:
# get directory path
path = str(pathlib.Path().cwd().parent.parent.resolve())

In [3]:
cpi = pd.read_csv(path + '/data/cpi.csv')
currency = pd.read_csv(path + '/data/currency_data.csv')
credit = pd.read_excel(path + '/data/credit.xlsx', header=1)
jse = pd.read_csv(path + '/data/jse_indices.csv')
fuel = pd.read_csv(path + '/data/fuel_df.csv').drop(['Unnamed: 0'], axis=1)
sabor = pd.read_csv(path + '/data/sabor.csv')

Change dates to datetime

In [4]:
# change month to datetime format
cpi['Date'] = pd.to_datetime(cpi['Month']).dt.strftime('%Y-%m')
currency['Date'] = pd.to_datetime(currency['Date']).dt.strftime('%Y-%m')
jse['Date'] = pd.to_datetime(jse['date']).dt.strftime('%Y-%m')
credit['date'] = pd.to_datetime(credit['Date'], format='%b, %Y')
credit['Date'] = pd.to_datetime(credit['date']).dt.strftime('%Y-%m')
fuel['Date'] = pd.to_datetime(fuel['date']).dt.strftime('%Y-%m')
sabor['Date'] = pd.to_datetime(sabor['date']).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_73039/602859498.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.


Only select dates from 2022

In [5]:
# change month to datetime format
cpi_new = cpi[cpi['Date'] > '2018']
currency_new = currency[currency['Date'] > '2018']
jse_new = jse[jse['Date'] > '2018']
credit_new= credit[credit['Date'] > '2018']
fuel_new= fuel[fuel['Date'] > '2018']
sabor_new= sabor[sabor['Date'] > '2018']

In [6]:
# Add June CPI
june_cpi = pd.DataFrame({'Category': ['Headline_CPI', 'Food and nonalcoholic beverages', 'Alcoholic beverages and tobacco', 
                                      'Clothing and footwear', 'Housing and utilities', 'Household contents and services',
                                      'Health', 'Transport', 'Communication', 'Recreation and culture', 'Education',
                                      'Restaurants and hotels ', 'Miscellaneous goods and services'], 
                         'Value': [109.8, 118.3, 110.9, 104.3, 105.4, 107.7, 110.8, 112.3, 99.6, 105.3, 110.4, 110.0, 109.6]})
june_cpi['Date'] = '2023-06'

In [7]:
cpi_new = pd.concat([cpi_new, june_cpi])
cpi_new['Date'] = pd.to_datetime(cpi_new['Date']).dt.strftime('%Y-%m')

In the credit and national excels the commas in values need to be stripped and values turned in to intergers

In [8]:
def remove_commas_and_convert_to_float(value):
    return float(value.replace(',', ''))

In [9]:
credit_new = credit_new.drop(['Share of corporations as a % of total credit',
       'Share of corporations as a % of total loans & advances',
       'Share of households as a % of total credit',
       'Share of households as % of total loans & advances', 'Investments'], axis=1)

In [10]:
credit_columns = ['Instalment sale credit', 'Leasing finance',
       'Mortgage advances', 'Overdrafts', 'General loans and advances',
       'Credit card advances', 'Of which: Total to households',
       'Total loans and advances : Households',
       'Claims on the domestic private sector',
       'Total loans and advances (excl. investments & bills)',
       'Bills discounted', 'Instalment sale credit.1', 'Leasing finance.1',
       'Mortgage advances.1', 'Overdrafts.1', 'General loans and advances.1',
       'Credit card advances.1', 'Of which: Total to corporations',
       'Claims on the domestic private sector.1',
       'Total loans and advances : Corporations',]

In [11]:
credit_new[credit_columns] = credit_new[credit_columns].applymap(remove_commas_and_convert_to_float)


# Create Features

In some of the excel sheets we do not have up to date. We will there use the feature from 2 months ao as the feature for now (ie march is the predictor for July)

In [12]:
print(jse_new['Date'].max())
print(credit_new['Date'].max())
print(currency_new['Date'].max())
print(cpi_new['Date'].max())
print(fuel_new['Date'].max())
print(sabor_new['Date'].max())

2023-06
2023-06
2023-06
2023-06
2023-07
2023-06


In [13]:
jse_new['newDate'] = (pd.to_datetime(jse_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
credit_new['newDate'] = (pd.to_datetime(credit_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
currency_new['newDate'] = (pd.to_datetime(currency_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
fuel_new['newDate'] = (pd.to_datetime(fuel_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')
sabor_new['newDate'] = (pd.to_datetime(sabor_new['Date']) + pd.DateOffset(months=2)).dt.strftime('%Y-%m')

/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_73039/1032560082.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/gx/c35dzbl95sg3t62y03g1j6s40000gn/T/ipykernel_73039/1032560082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
jse_new = jse_new.drop(['Date', 'date'], axis=1)
credit_new = credit_new.drop(['Date', 'date'], axis=1)
currency_new = currency_new.drop(['Date'], axis=1)
fuel_new = fuel_new.drop(['Date', 'date'], axis=1)
sabor_new = sabor_new.drop(['Date', 'date'], axis=1)

## Get mean per Month for jse and currency

In [15]:
credit_mean = credit_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
jse_mean = jse_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
currency_mean = currency_new.groupby(['newDate']).mean().reset_index().add_suffix("mean")
fuel_new = fuel_new.groupby(['newDate']).mean().reset_index()
sabor_new = sabor_new.groupby(['newDate']).mean().reset_index()


jse_std = jse_new.groupby(['newDate']).std().reset_index().add_suffix("std")
currency_std = currency_new.groupby(['newDate']).std().reset_index().add_suffix("std")

credit_min = credit_new.groupby(['newDate']).min().reset_index().add_suffix("min")
jse_min = jse_new.groupby(['newDate']).min().reset_index().add_suffix("min")
currency_min = currency_new.groupby(['newDate']).min().reset_index().add_suffix("min")

credit_max = credit_new.groupby(['newDate']).max().reset_index().add_suffix("max")
jse_max = jse_new.groupby(['newDate']).max().reset_index().add_suffix("max")
currency_max = currency_new.groupby(['newDate']).max().reset_index().add_suffix("max")


In [16]:
credit_new = credit_mean.merge(credit_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
credit_new = credit_new.merge(credit_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
credit_new = credit_new.rename(columns={"newDatemean": "newDate"})

In [17]:
jse_new = jse_mean.merge(jse_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
jse_new = jse_new.merge(jse_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
jse_new = jse_new.merge(jse_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
jse_new = jse_new.rename(columns={"newDatemean": "newDate"})

In [18]:
currency_new = currency_mean.merge(currency_std, left_on='newDatemean', right_on='newDatestd', how='left').drop(['newDatestd'], axis=1)
currency_new = currency_new.merge(currency_max, left_on='newDatemean', right_on='newDatemax', how='left').drop(['newDatemax'], axis=1)
currency_new = currency_new.merge(currency_min, left_on='newDatemean', right_on='newDatemin', how='left').drop(['newDatemin'], axis=1)
currency_new = currency_new.rename(columns={"newDatemean": "newDate"})

# Merge data together for features

In [19]:
cpi_all = cpi.drop(['Month'], axis=1).merge(jse_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(credit_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(currency_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(fuel_new, right_on='newDate', left_on='Date', how='left')
cpi_all = cpi_all.drop(['newDate'], axis=1).merge(sabor_new, right_on='newDate', left_on='Date', how='left')

In [20]:
cpi_all.shape

(221, 103)

# Model
We will model one index at a time to see what works the best

In [21]:
from models.models import HoltWintersWrapper, ProphetWrapper
from sklearn.metrics import mean_squared_error

In [25]:
feature_columns = list(cpi_all.drop(['Category', 'Date', 'Value', 'newDate'], axis=1).columns)

In [329]:
feature_columns

['Percentage Change (From Prior Month)',
 'Top 40 Indexmean',
 'Mid Cap Indexmean',
 'Small Cap Indexmean',
 'All-Share Indexmean',
 'Large Cap Indexmean',
 'Fin 15 Indexmean',
 'Top 40 Indexstd',
 'Mid Cap Indexstd',
 'Small Cap Indexstd',
 'All-Share Indexstd',
 'Large Cap Indexstd',
 'Fin 15 Indexstd',
 'Top 40 Indexmax',
 'Mid Cap Indexmax',
 'Small Cap Indexmax',
 'All-Share Indexmax',
 'Large Cap Indexmax',
 'Fin 15 Indexmax',
 'Top 40 Indexmin',
 'Mid Cap Indexmin',
 'Small Cap Indexmin',
 'All-Share Indexmin',
 'Large Cap Indexmin',
 'Fin 15 Indexmin',
 'Instalment sale creditmean',
 'Leasing financemean',
 'Mortgage advancesmean',
 'Overdraftsmean',
 'General loans and advancesmean',
 'Credit card advancesmean',
 'Of which: Total to householdsmean',
 'Total loans and advances : Householdsmean',
 'Claims on the domestic private sectormean',
 'Total loans and advances (excl. investments & bills)mean',
 'Bills discountedmean',
 'Instalment sale credit.1mean',
 'Leasing finance.1m

In [342]:
def combined_model(cpi_all, month):

    hw_cpi_cat = ['Health', 'Education']
    cpi_cat = ['Headline_CPI',
                'Food and non-alcoholic beverages',
                'Alcoholic beverages and tobacco',
                'Clothing and footwear',
                'Housing and utilities',
                'Household contents and services',
                'Transport',
                'Communication',
                'Recreation and culture',
                'Restaurants and hotels ',
                'Miscellaneous goods and services']

    hw_6_results = []
    prophet_multi = []

    # fit hw
    for cat in hw_cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        hw_6 = HoltWintersWrapper(seasonal_periods=6)

        hw_6.fit(y=df[df['Date'] < month].sort_values(by='Date')['Value'].values)

        hw_6_results.append(hw_6.predict(forcast=1)[0])

    df_hw_results = pd.DataFrame({'cat':hw_cpi_cat, 'pred':hw_6_results})

    columns_to_drop = ['Category', 'Percentage Change (From Prior Month)', 'newDate']

    df_features = jse_new.merge(credit_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(currency_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(fuel_new, right_on='newDate', left_on='newDate', how='left')
    df_features = df_features.merge(sabor_new, right_on='newDate', left_on='newDate', how='left')

    for cat in cpi_cat:
        df = cpi_all[cpi_all['Category'] == cat]

        # reorder and drop columns
        df.insert(0, 'ds', df.pop('Date'))
        df.insert(1, 'y', df.pop('Value'))
        df = df[['ds', 'y', 'sabor']]

        # model
        prophet = ProphetWrapper(n_changepoints=2, seasonality_mode="multiplicative")
        prophet.fit(df[df['ds'] < month].sort_values(by='ds').sort_values(by='ds'))

        df_predict = pd.DataFrame({'ds': [month]})
        df_predict['ds'] = pd.to_datetime(df_predict['ds']).dt.strftime('%Y-%m')

        df_predict = df_predict.merge(df_features, left_on='ds', right_on='newDate').drop(['newDate'], axis=1)
        df_predict_columns = df.drop(['y'], axis=1).columns
        prophet_multi.append(prophet.predict(df_predict[df_predict_columns])[0])

    df_prophet_results = pd.DataFrame({'cat':cpi_cat, 'pred':prophet_multi})

    df_results = pd.concat([df_hw_results, df_prophet_results])

    return df_results

        

## Get April predictions

In [343]:
df_results_apr = combined_model(cpi_all=cpi_all, month='2023-04')

In [344]:
cpi_apr = cpi_all[cpi_all['Date'] == '2023-04']

In [345]:
df_results_apr = df_results_apr.merge(cpi_apr[['Category', 'Value']], left_on='cat', right_on='Category')

In [346]:
df_results_apr

,cat,pred,Category,Value
0,Health,110.219044,Health,109.6
1,Education,110.397182,Education,110.4
2,Headline_CPI,109.122554,Headline_CPI,109.4
3,Food and non-alcoholic beverages,118.048220,Food and non-alcoholic beverages,117.4
4,Alcoholic beverages and tobacco,109.497112,Alcoholic beverages and tobacco,110.2
5,Clothing and footwear,103.585793,Clothing and footwear,103.7
6,Housing and utilities,104.672730,Housing and utilities,104.6
7,Household contents and services,108.178676,Household contents and services,107.7
8,Transport,112.570759,Transport,113.1
9,Communication,99.989488,Communication,99.8


In [347]:
rmse_apr = mean_squared_error(df_results_apr['pred'], df_results_apr['Value'], squared=False)
print('Apr RMSE prophet multi: ', rmse_apr)

Apr RMSE prophet multi:  0.5732737122456786


In [348]:
df_results_apr = df_results_apr[['cat', 'pred']]
df_results_apr.columns = ['ID', 'Value']

month = "April"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_apr = df_results_apr.replace(pred_map)

df_results_apr.to_csv(path + '/submissions/apr_testing.csv', index=False)

## Get May predictions

In [349]:
df_results_may = combined_model(cpi_all=cpi_all, month='2023-05')

In [350]:
cpi_may = cpi_all[cpi_all['Date'] == '2023-05']

In [351]:
df_results_may = df_results_may.merge(cpi_may[['Category', 'Value']], left_on='cat', right_on='Category')

In [352]:
df_results_may

,cat,pred,Category,Value
0,Health,109.862401,Health,110.3
1,Education,110.399999,Education,110.4
2,Headline_CPI,109.928225,Headline_CPI,109.6
3,Food and non-alcoholic beverages,118.953894,Food and non-alcoholic beverages,117.7
4,Alcoholic beverages and tobacco,111.557328,Alcoholic beverages and tobacco,110.6
5,Clothing and footwear,103.766891,Clothing and footwear,104.1
6,Housing and utilities,104.895073,Housing and utilities,104.6
7,Household contents and services,108.393757,Household contents and services,107.5
8,Transport,115.092887,Transport,113.3
9,Communication,100.070658,Communication,99.7


In [353]:
rmse_may = mean_squared_error(df_results_may['pred'], df_results_may['Value'], squared=False)
print('May RMSE prophet multo: ', rmse_may)

May RMSE prophet multo:  0.8014157441626073


In [354]:
df_results_may = df_results_may[['cat', 'pred']]
df_results_may.columns = ['ID', 'Value']

month = "May"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_may = df_results_may.replace(pred_map)

df_results_may.to_csv(path + '/submissions/may_testing.csv', index=False)

## Get June predictions

In [355]:
df_results_june = combined_model(cpi_all=cpi_all, month='2023-06')

In [356]:
df_results_june['Value'] = [110.8, 110.4, 109.8, 118.3, 110.9, 104.3, 105.4, 107.7, 112.3, 99.6, 105.3, 110.0, 109.6]

In [357]:
rmse_June = mean_squared_error(df_results_june['pred'], df_results_june['Value'], squared=False)
print('June RMSE prophet multo: ', rmse_June)

June RMSE prophet multo:  0.6928484695650284


In [358]:
df_results_june

,cat,pred,Value
0,Health,110.510737,110.8
1,Education,110.400154,110.4
0,Headline_CPI,110.506180,109.8
1,Food and non-alcoholic beverages,119.329848,118.3
2,Alcoholic beverages and tobacco,111.549531,110.9
3,Clothing and footwear,104.247566,104.3
4,Housing and utilities,104.730566,105.4
5,Household contents and services,108.217005,107.7
6,Transport,112.119872,112.3
7,Communication,99.929319,99.6


In [328]:
df_results_june = df_results_june[['cat', 'pred']]
df_results_june.columns = ['ID', 'Value']

month = "June"

pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

df_results_june = df_results_june.replace(pred_map)

df_results_june.to_csv(path + '/submissions/june_testing.csv', index=False)

# Get predictions for July

In [ ]:
df_results_july = combined_model(cpi_all=cpi_all, month='2023-07')

In [ ]:
df_results_july.columns = ['ID', 'Value']

In [ ]:
month = 'July'

In [ ]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [ ]:
df_results_july = df_results_july.replace(pred_map)

In [ ]:
df_results_july.to_csv(path + '/submissions/2023-08-13_multi_july.csv', index=False)

# Get August submission

In [ ]:
df_results_aug = combined_model(cpi_all=cpi_all, month='2023-08')

In [ ]:
df_results_aug.columns = ['ID', 'Value']

In [ ]:
month = 'August'

In [ ]:
pred_map = {'Headline_CPI': f'{month}_headline CPI',
'Alcoholic beverages and tobacco': f'{month}_alcoholic beverages and tobacco',
'Clothing and footwear': f'{month}_clothing and footwear',
'Communication': f'{month}_communication',
'Education': f'{month}_education',
'Food and non-alcoholic beverages': f'{month}_food and non-alcoholic beverages',
'Health': f'{month}_health',
'Household contents and services': f'{month}_household contents and services',
'Housing and utilities': f'{month}_housing and utilities',
'Miscellaneous goods and services': f'{month}_miscellaneous goods and services',
'Recreation and culture': f'{month}_recreation and culture',
'Restaurants and hotels ': f'{month}_restaurants and hotels',
'Transport': f'{month}_transport'}

In [ ]:
df_results_aug = df_results_aug.replace(pred_map)

In [ ]:
df_results_aug.to_csv(path + '/submissions/2023-08-13_multi_aug.csv', index=False)